In [9]:
import ipywidgets as widgets
from IPython.display import display, HTML
from fractions import Fraction

# NOTES
note_durations = [
    ("Whole Note",         Fraction(1)),
    ("Dotted Half",        Fraction(3, 4)),
    ("Half Note",          Fraction(1, 2)),
    ("Triplet Half",       Fraction(1, 3)),
    ("Dotted Quarter",     Fraction(3, 8)),
    ("Quarter Note",       Fraction(1, 4)),
    ("Triplet Quarter",    Fraction(1, 6)),
    ("Dotted 8th",         Fraction(3, 16)),
    ("8th Note",           Fraction(1, 8)),
    ("Triplet 8th",        Fraction(1, 12)),
    ("Dotted 16th",        Fraction(3, 32)),
    ("16th Note",          Fraction(1, 16)),
    ("Triplet 16th",       Fraction(1, 24)),
    ("Dotted 32nd",        Fraction(3, 64)),
    ("32nd Note",          Fraction(1, 32)),
    ("Triplet 32nd",       Fraction(1, 48)),
    ("Dotted 64th",        Fraction(3, 128)),
    ("64th Note",          Fraction(1, 64)),
]

# GLOBAL SLIDERS
tempo_slider = widgets.IntSlider(description='Tempo', min=30, max=300, step=1, value=90)
signum_slider = widgets.IntSlider(description='Top Time Sig', min=2, max=15, step=1, value=4)
signote_slider = widgets.SelectionSlider(description='Btm Time Sig', options=[4, 8, 16], value=4)
division_slider = widgets.SelectionSlider(description='Max Division', options=[8, 16, 32, 12, 24, 48], value=8)
measures_slider = widgets.IntSlider(description='Measures', min=1, max=12, step=1, value=1)

# TRACK SPECIFIC SLIDERS - HELPER
def make_param_sliders(name=''):
    return [
        widgets.SelectionSlider(description=f'{name}Note', options=note_durations, value=Fraction(1, 4),
                                layout={'width': '300px'}, style={'description_width': '80px'}),
        widgets.IntSlider(description=f'{name}Offset', min=0, max=64, step=1, value=0),
        widgets.IntSlider(description=f'{name}Group', min=0, max=100, step=1, value=0),
        widgets.IntSlider(description=f'{name}Param4', min=0, max=100, step=1, value=0),
    ]

# Create slider groups for H, S, K
h_params = make_param_sliders("Hi Hat-")
s_params = make_param_sliders("Snare-")
k_params = make_param_sliders("Kick-")

sliders = [
    tempo_slider, signum_slider, signote_slider, division_slider, measures_slider,
    *h_params, *s_params, *k_params
]

output = widgets.Output()

# PATTERN GENERATOR
def generate_blank_measure(signum, signote, division, tempo=None, measures=1, p1=None, p2=None, p3=None, p4=None):
    binary_length = int((signum * division) / signote)
    pattern = ['0'] * binary_length

    # Note Injector
    if p1 and p1 > 0:
        steps_per_hit = int(division * p1)
        if steps_per_hit > 0:
            for i in range(0, binary_length, steps_per_hit):
                pattern[i] = '1'

    # Offset
    if p2 > 0:
        pattern = (['0'] * p2) + pattern[:-p2]
        if p2 > binary_length:
            pattern = pattern[:-(p2 - binary_length)]

    # Repeat for measures
    pattern = pattern * measures
    return pattern

# UPDATER
def on_slider_change(change=None):
    with output:
        output.clear_output()

        # Get values
        signum = signum_slider.value
        signote = signote_slider.value
        division = division_slider.value
        tempo = tempo_slider.value
        measures = measures_slider.value

        # Get parameters
        def get_params(param_group):
            return [w.value for w in param_group]

        p_h = get_params(h_params)
        p_s = get_params(s_params)
        p_k = get_params(k_params)

        # Generate patterns
        def to_pretty(binary, char='o'):
            return ''.join(binary).replace('1', char).replace('0', '-')

        h_bin = generate_blank_measure(signum, signote, division, tempo, measures, *p_h)
        s_bin = generate_blank_measure(signum, signote, division, tempo, measures, *p_s)
        k_bin = generate_blank_measure(signum, signote, division, tempo, measures, *p_k)

        h_conv = to_pretty(h_bin, 'x')  # x for hi-hat normal  (options: x, o, X, c, r, b, m, s, n, N)
        s_conv = to_pretty(s_bin, 'o')  # o for snare normal   (options: o, O, g, x, b, f)
        k_conv = to_pretty(k_bin, 'o')  # o for kick normal    (options: o, x, X)

        # BINARY
        print(f"H: {''.join(h_bin)}")
        print(f"S: {''.join(s_bin)}")
        print(f"K: {''.join(k_bin)}")

        # GROOVE
        print(f"H: {h_conv}")
        print(f"S: {s_conv}")
        print(f"K: {k_conv}")

        # HEX
        try:
            hex_h = f"{int(''.join(h_bin), 2):x}".upper()
            hex_s = f"{int(''.join(s_bin), 2):x}".upper()
            hex_k = f"{int(''.join(k_bin), 2):x}".upper()
            print(f"Hex H: {hex_h}")
            print(f"Hex S: {hex_s}")
            print(f"Hex K: {hex_k}")
        except:
            print("Error converting to hex.")

        # GENERATE LINK
        groove_url = (
            f"https://www.mikeslessons.com/groove/?"
            f"TimeSig={signum}/{signote}&"
            f"Div={division}&"
            f"Tempo={tempo}&"
            f"Measures={measures}&"
            f"H=|{h_conv}|&"
            f"S=|{s_conv}|&"
            f"K=|{k_conv}|"
        )

        print("\nGrooveScribe Link:")
        display(HTML(f'<a href="{groove_url}" target="_blank">LINK TO PLAY/EDIT</a>'))
        print(groove_url)

# OBSERVE!
for s in sliders:
    s.observe(on_slider_change, names='value')

# UI
display(widgets.VBox(sliders))
display(output)

# Trigger on load
on_slider_change()


Output()